In [20]:
#<imports>
import os
from getpass import getpass

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFaceHub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
#<\imports>

In [2]:
#<get hf token>
HF_TOKEN = getpass("Huggingface Token : ")
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_TOKEN
#<\get hf token>

In [6]:
#<load document>
loader = PyPDFLoader("data/sample_data.pdf")
data = loader.load()
#<\load document>

In [9]:
#<make chunks>
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 512,chunk_overlap = 0)
chunks = text_splitter.split_documents(data)
#<\make chunks>

In [ ]:
chunks[0]

In [ ]:
#<make embeddings>
model_name = "thenlper/gte-large"
embedding_model = FastEmbedEmbeddings(model_name=model_name)
#<\make embeddings>

In [14]:
#<make chroma db>
db = Chroma.from_documents(chunks, embedding_model, persist_directory="./chroma_db")
#<make chroma db>

In [15]:
#<define query that is used multiple times>
query = "what happens if I sell my dreams?"
#<\define query that is used multiple times>

In [ ]:
#<retrieve docs>
db2 = Chroma(persist_directory="./chroma_db", embedding_function=embedding_model)
retrieved_docs = db2.similarity_search(query)
print(retrieved_docs[0].page_content)
#<\retrieve docs>

In [17]:
#<initialize retriever>
retriever = db2.as_retriever(
    search_type="mmr", #similarity
    search_kwargs={'k': 4}
    )
#<\initialize retriever>

In [ ]:
#<define llm>
llm = HuggingFaceHub(repo_id="mistralai/Mistral-7B-Instruct-v0.2",
                     model_kwargs={
                         "temperature":0.1,
                         "max_new_tokens":512,
                         "return_full_text":False,
                         "repetition_penalty":1.1,
                         "top_p":0.9
                         }
                    )
#<\define llm>

In [21]:
#<define chain components and chain>
#template
template = """
<s>[INST]
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT
[/INST]
CONTEXT: {context}
</s>
[INST]
{query}
[/INST]
"""
#prompt
prompt = ChatPromptTemplate.from_template(template)

#output parser
output_parser = StrOutputParser()

#chain
chain = (
{"context": retriever, "query": RunnablePassthrough()}
| prompt
| llm
| output_parser
)

#<\define chain components and chain>

In [ ]:
#<test response>
response = chain.invoke(query)
response
#<\test response>

In [ ]:
response2 = chain.invoke("what is the authors name?")
response2

In [ ]:
response3 = chain.invoke("For how many years does the author tell the lady to leave right away and not come back to Vienna")
response3